## Self Healing Airy Beam

### 1. What is an Airy Beam?

An Airy beam is a special type of non-diffracting optical beam that exhibits unique properties, such as **self-acceleration, self-healing, and non-spreading propagation**. It was first theoretically proposed by physicist Michael Berry and colleagues in 1979 and experimentally demonstrated in 2007.

Comparison between *Airy* beam and *Gaussian* beam :
| **Property**               | **Airy Beam**                         | **Gaussian Beam**                      |
|----------------------------|---------------------------------------|----------------------------------------|
| **Intensity Profile**      | Asymmetric (Airy function + side lobes) | Symmetric (bell-shaped, no side lobes) |
| **Propagation Path**       | Curved (self-accelerating)            | Straight                               |
| **Diffraction**            | Resists spreading (non-diffracting)   | Spreads due to diffraction             |
| **Self-Healing**           | Yes (recovers after obstacles)        | No (blocked light is lost)             |
| **Energy Distribution**    | Infinite (ideal) / Finite (apodized)  | Always finite                          |
| **Applications**           | Optical trapping, curved plasmons     | Lasers, imaging, communications        |
| **Mathematical Form**      | $Ai(x) \cdot e^{a x}$ (apodized)    | $e^{-x^2/w^2}$ (Gaussian)           |

---

### 2. Formula for Airy Beam

Formula for a 2-D nonspreading Airy beam.

$$
F(x,y,z)=Ai\left(\frac{x}{x_0}\right)Ai\left(\frac{y}{y_0}\right)e^{a_1x+a_2y}
$$
$$
\begin{align*}
\text{where}\quad x_0&=\text{scale }x\\
y_0&=\text{scale }y\\
a_1&=\text{degree of apodization in }x\\
a_2&=\text{degree of apodization in }y\\
\end{align*}
$$
- **Apodization factor** $a_1$ and $a_2$ are introduced to truncate the Airy beam, making it physically realizable.
- The most common form is an exponential apodization, applied to the transverse coordinates $x$ and $y$.
**Apodization** refers to :

> Process of modifying the beam's infinite-energy ideal profile into a finite-energy version by applying an exponential or Gaussian decay factor.

This is necessary because a true (mathematically ideal) Airy beam has infinite energy due to its slowly decaying side lobes, making it unrealizable in experiments.

---

### 3. Role of Apodization

The role of Apodization $a_1$ and $a_2:$ 

1. Controls Energy Finiteness
    - A small $a$ (e.g., $a=0.1$) ensures the beam remains close to an ideal Airy beam but with finite energy.
    - Larger $a$ truncates the beam more aggressively, reducing side lobes but also weakening non-diffracting properties.

2. Affects Beam Propagation
    - Higher $a$ reduces the self-accelerating effect and self-healing capability.
    - Too small $a$ makes the beam experimentally unrealizable (still too much energy in side lobes).

3. Balances Between Ideal and Realizable
    - A typical experimental choice is $a\approx0.05−0.2.$


Comparison between *Ideal* and *Apodized* Airy beam :
| **Property**               | **Ideal Airy Beam $(a = 0)$**       | **Apodized Airy Beam $(a > 0)$**      |
|----------------------------|---------------------------------------|-----------------------------------------|
| **Energy**                 | Infinite (unphysical)                 | Finite (experimentally realizable)      |
| **Side Lobes**             | Strong, slowly decaying (∼1/𝑥)        | Exponentially suppressed $(\approx e^{ax})$       |
| **Self-Healing**           | Perfect (theoretically)               | Limited (depends on $a$)             |
| **Self-Acceleration**      | Fully preserved                       | Reduced with larger $a$              |
| **Diffraction Resistance** | Strong (non-diffracting)              | Weakened (more diffraction over distance) |
| **Experimental Feasibility** | Impossible (infinite energy)        | Achievable (with spatial light modulators) |

---

### 4. Simulation

Below is a simulation of an *Apodized Airy beam* which **passes through an object**.

Reference: https://opticspy.github.io/lightpipes/2DAiryBeam.html#self-healing-airy-beam

In [ ]:
"""
Self healing Airy beam. A disk is placed at some distance from the origin.
This obstacle disturbs the beam, but it heals itself.
"""

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from LightPipes import *

# Parameters
labda = 2.3*um      # Wavelength
size = 30*mm        # Grid size
N = 500             # Number of grids
x0 = y0 = 1*mm      # Scale
a1 = a2 = 0.1/mm    # Apodization
prop = 2000*cm      # Total desired propagation distance
z = 0*cm            # Current propagation distance
dz = 2*cm           # Propagation step size

# Obstacle Parameters
obs1 = 300*cm        # Distance of onstacle from starting point
r1 = 3*mm            # Radius of circular screen
activated1 = False   # Checks whether beam has encountered object

obs2 = 800*cm        # Distance of onstacle from starting point
r2 = 1*mm            # Radius of circular screen
activated2 = False   # Checks whether beam has encountered object

# Create Matplotlib Figure and Axes
ims = [] # List of lists for Artist(s) during Animation. e.g. [[A1,A2], [A1,A2]]
fig, ax = plt.subplots()

# Creates beam
F0 = Begin(size, labda, N)          # Initialises beam
F0 = AiryBeam2D(F0, x0, y0, a1, a2) # Converts beam to Airy beam
# F0 = GaussBeam(F0, x0)


# Simulate propagation
for i in range(int(prop/dz)):
    # Obstacle 1
    if z >= obs1 and not activated1:
        F0 = CircScreen(F0, r1)
        activated1 = True

    # Obstacle 2
    if z >= obs2 and not activated2:
        F0 = CircScreen(F0, r2)
        activated2 = True

    # Frame by frame Intensity of Airy beam
    F = Fresnel(F0, z) # Propgation of field using convolution method
    I = Intensity(F)
    im = ax.imshow(I, animated=True, cmap='jet') # Frame
    t = ax.annotate(text="z = {:4.0f}cm".format(z/cm), xy=(100,100), color='w') # Annotate distance propagated, z

    ims.append([im, t])

    # Update current propagation distance
    z += dz 

ani = animation.ArtistAnimation(fig, ims, interval=5, blit=True, repeat_delay=1000)
ani.save("airy-beam-2obstacle-3mmmm-at300cm600cm.mp4")
plt.close() # Hide extra image